In [1]:
import numpy as np
import matplotlib.pylab as plt
import cv2
import os
import random
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
x_npz = np.load("images.npz")
x = x_npz['arr_0']
y_npz = np.load("labels.npz")
y = y_npz['arr_0']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=1)
print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)

(60, 256, 320, 3)
(20, 256, 320, 3)
(60, 256, 320)
(20, 256, 320)


In [ ]:
# Matplotlib black magic
plt.figure(figsize=(10,10))
plt.imshow(X_train[random.randint(0,len(X_train))])  

In [26]:
n_train_samples, img_height, img_width, n_channels = np.array(X_train).shape

In [28]:
n_train_samples, img_height, img_width, n_channels = np.array(X_train).shape
n_classes = 11

n_test_samples = len(X_test)

t = np.zeros((n_train_samples, img_height, img_width, n_classes))

for i in range (0, img_height):
    for j in range (0, img_width):
        for k in range (0, n_train_samples):
            t[k][i][j][y_train[k][i][j] - 1] = 1
            
y_train = t
del(t)

In [86]:
print(y_train.shape)

(60, 256, 320, 11)


In [35]:
# UNET-11 class model
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

def Unet11(input_size = (256,320,3), n_classes = 11):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(n_classes, 1, activation = 'softmax')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

#     model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
#     if(pretrained_weights):
#         model.load_weights(pretrained_weights)

    return model

In [37]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255, horizontal_flip=True, rotation_range=30)
test_datagen = ImageDataGenerator(rescale=1. / 255)

batch_size = 4

train_generator = train_datagen.flow(np.array(X_train), y_train, batch_size=batch_size)
test_generator = test_datagen.flow(np.array(X_test), y_test, batch_size=batch_size)

In [43]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

model = Unet11()

weights_path = 'weight.h5'

epochs_arr  = [   20,      5,      5]
learn_rates = [0.001, 0.0005, 0.0001]

for learn_rate, epochs in zip(learn_rates, epochs_arr):
#     if os.path.isfile(weights_path):
#         print("loading existing weight for training")
#         model.load_weights(weights_path)
    
    opt  = optimizers.Adam(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                  optimizer=opt,
                  metrics=['accuracy'])
    callbacks = [ModelCheckpoint(weights_path, monitor='loss', save_best_only=True, verbose=2)] #TODO

#     model.fit(x = X_train, y= Y_train, validation_data=(X_val, Y_val),
#           batch_size=256, verbose=2, epochs=epochs, callbacks=callbacks, shuffle=True)

    history = model.fit_generator(train_generator, steps_per_epoch=n_train_samples // batch_size, epochs=epochs,
                                 verbose=1, callbacks = callbacks)
    
model.save_weights('weights.h5')


Epoch 1/20


D:\Anaconda\lib\site-packages\keras\callbacks.py:496: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
D:\Anaconda\lib\site-packages\keras\callbacks.py:405: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


57s - loss: 2.7786 - acc: 0.5386
Epoch 2/20
38s - loss: 2.3747 - acc: 0.6146
Epoch 3/20
38s - loss: 2.3581 - acc: 0.6146
Epoch 4/20
39s - loss: 2.3416 - acc: 0.6146
Epoch 5/20
39s - loss: 2.3253 - acc: 0.6146
Epoch 6/20
39s - loss: 2.3093 - acc: 0.6146
Epoch 7/20
39s - loss: 2.2932 - acc: 0.6146
Epoch 8/20
39s - loss: 2.2772 - acc: 0.6146
Epoch 9/20
39s - loss: 2.2617 - acc: 0.6146
Epoch 10/20
39s - loss: 2.2461 - acc: 0.6146
Epoch 11/20
39s - loss: 2.2305 - acc: 0.6146
Epoch 12/20
39s - loss: 2.2154 - acc: 0.6146
Epoch 13/20
39s - loss: 2.2004 - acc: 0.6146
Epoch 14/20
39s - loss: 2.1855 - acc: 0.6146
Epoch 15/20
40s - loss: 2.1707 - acc: 0.6146
Epoch 16/20
39s - loss: 2.1562 - acc: 0.6146
Epoch 17/20
39s - loss: 2.1418 - acc: 0.6146
Epoch 18/20
39s - loss: 2.1276 - acc: 0.6146
Epoch 19/20
39s - loss: 2.1134 - acc: 0.6146
Epoch 20/20
40s - loss: 2.0996 - acc: 0.6146
Epoch 1/5
40s - loss: 2.0907 - acc: 0.6281
Epoch 2/5
39s - loss: 2.0740 - acc: 0.6202
Epoch 3/5
40s - loss: 2.0875 - acc

In [44]:
# model.save_weights('Unet11weights.h5')

In [76]:
results = model.predict_generator(test_generator,4, verbose=1)

4/4 [==============================] - 2s     


In [79]:
results[0][0,100,:]

array([ 0.1238859 ,  0.14719352,  0.07616436,  0.1073738 ,  0.08353125,
        0.07667171,  0.07949865,  0.07566057,  0.07565576,  0.07574201,
        0.07862248], dtype=float32)

In [ ]:
t = np.zeros((n_train_samples, img_height, img_width, n_classes))

for i in range (0, img_height):
    for j in range (0, img_width):
        for k in range (0, n_train_samples):
            t[k][i][j][y_train[k][i][j] - 1] = 1